# Tool for valid launch times of a satellite A to target specific orbit plane to trail RAAN of a satellite B 

Author: Andrew Ng, 16/11/2022

Code flow:
1. Pull TLE data from Celestrak
2. Propagate target satellite B's RAAN for specified time with J2 effect
3. Obtain RAAN target plane for satellite A to launch into 
4. Match RAAN to longitude of launch site within tolerance, correcting longitude with the angle between vernal equinox and prime meridian. 
5. Obtain time stamps when condition in step 4 are valid. These are valid launch times subject to the tolerance chosen. 


Required inputs:
data_propagate_J2_list - list of strings, in form "YYYYMMDD". This is your target launch date. There will be 1 launch window per day.
cat_id - int, NORAD ID of satellite. NEUSAR--> 52937

### Output of this code: 
2 csv files: "valid_launch_times_jupyter.csv" and "output_data_jupyter.csv". 

The former contains valid launch times matching the RAAN condition for T2, along with the uncertainty in RAAN purely due to the uncertainity in launch azimuth/ launch sequence which are not provided by ISRO. 

The latter contains data for all propagated timesteps. 

First install dependencies.

In [11]:
import sys
%conda install --yes --prefix {sys.prefix} numpy


Note: you may need to restart the kernel to use updated packages.



EnvironmentLocationNotFound: Not a conda environment: C:\UsersNgengtonAnaconda3



In [12]:
from propagate_J2 import propagate_J2

data_propagate_J2_list = ["20230422"]
cat_id = 52937  #NEUSAR
use_today_date = True



# Propagate orbit plane's RAAN with J2 effect
Using the most current TLE from Celestrak, we output the RAAN due to J2 on a date in the future. 

The rate of change of RAAN with time: 
\begin{equation}
\frac{d\Omega}{dt} = -\frac{3}{2}J_2(\frac{R_E^2}{a^2(1-e^2)})^2\times \tilde{n}\cos{i}
\end{equation}

$R_E$ denotes the radius of Earth (km)  
$\mu_E$ denotes the gravitational constant of the earth  
$\tilde{n}$ denotes the perturbed mean motion. Some authors use mean motion rather than its perturbed version for simplicity, however this introduces errors over longer periods of time. Using the perturbed mean motion also gives a closer value to HPOP, which serves as a ground truth. (K.F. Wakker, Page 631, Fundamentals of Astrodynamics, TU Delft 2015)

The perturbed mean motion is related to the mean motion $n$, via: 
\begin{equation}
\tilde{n} = n [1 + \frac{3}{4}J_2(\frac{R}{a})^2(1-e^2)^(-3/2)(3\cos(i)^2-1)]
\end{equation}


Lastly, the final RAAN value is corrected by including a linear interpolation of the deviation between the J2 propagator and HPOP from STK. This is done by precomputing deviations over days between TLE and prediction epoch. HPOP models all gravitational harmonics and perturbing forces, and because of this, the RAAN precession effect is greater than the J2 analytical method we are using. This difference is determinisitic for our use case, so we proceed by using the interpolation to correct for our RAAN value. 

\begin{equation}
deviation = {RAAN}_{HPOP}-{RAAN}_{J2}
\end{equation}

<img src="deviation_between_j2_hpop.png" />

# Obtaining target orbit plane

We are interested in launching our satellite into a target plane, displaced by $120\degree$ from NeuSAR. This is done by simply subtracting $120\degree$ from NeuSAR's RAAN. I.e
\begin{equation}
RAAN_{T2} = RAAN_{NeuSAR} - 120\degree
\end{equation}

In [13]:
overall_df = propagate_J2(data_propagate_J2_list, cat_id, use_today_date)

d:\GitHub\TLE_pull\pulled_data\data_20230228\230228_NEUSAR_52937.tle
Inclination of orbit is: 9.985


# Valid launch times
Final goal is to get valid times to launch. Since the launch site is fixed, and we know the target orbit plane we require and how it evolves over time, we are able to then match the target orbit plane's RAAN with the $RAAN_{launched}$, the RAAN from launching satellite into orbit at a certain time. The time of day and date affects the $RAAN_{launched}$, so the launch window is dependent on both the date and time of day.  

Relation between $RAAN_{launched}$ and RAAN is via the greenwich apparent sidereal angle (GAST)

<img src="Sidereal_Time_Fig_2.png" />

$RAAN_{launched} = GAST + \text{launch site longitude accounting for launch azimuth}$

With a variable del_RAAN to set the tolerance for the orbit plane, which accounts for uncertainties such as the launch trajectory, we can select valid launch times where the target RAAN matches the LAN within tolerance. These then becomes the valid launch times.  

It is recommended for del_RAAN $ > 4$ as the launch duration is slated to be 20mins +, which translates to a rotation of about $2.5\degree$ in longitude from the time of launch to end.  



In [14]:
from propagate_J2 import output_launch_times

RAAN_tol = 5 #degrees
launch_site_coords = (13.73204, 80.23621)
launch_azimuth = 102 #degree
launch_data = output_launch_times(overall_df, launch_site_coords, RAAN_tol)
valid_launch_times = launch_data["T_prop_T2 (UTC)"]



Output data to .csv files. 

In [15]:
overall_df.to_csv("output_data_jupyter.csv")
launch_data.to_csv("valid_launch_times_jupyter.csv", index = False)